In [172]:
import csv
import numpy as np

In [36]:
with open('JRcleaningcontinued.csv', 'rU') as csvfile:
    reader = csv.DictReader(csvfile)
    crimeDictionary = {}
    for row in reader:
        key = (int(row['EPOCH_TRUNC']), (float(row['LATITUDE'][:4]), float(row['LONGITUDE'][:5])))
        if key in crimeDictionary:
            crimeDictionary[key].append(row)
        else:
            crimeDictionary[key] = [row]
        

In [37]:
len(crimeDictionary[crimeDictionary.keys()[4]])

4

In [38]:
import pickle
weatherFyle = open('JRcompleteweatherdata.pkl')
weatherDF = pickle.load(weatherFyle)

In [39]:
#minTime = 1341115200, July 2012
#maxTime = 1443495600, Sept 29, 2015
#LATITUDE
#LONGITUDE
#time
#temperature
#pressure
#humidity
#visibility
#precipIntensity
#precipProbability
weatherDict = {}

for x in weatherDF.iterrows():
    clock = float(str(x[0]).split(' ')[1][0:2])
    key = (x[1]['time'], (x[1]['LATITUDE'], x[1]['LONGITUDE']))
    weatherDict[key] = {'clock': clock, 'temp': x[1]['temperature'], 'pressure': x[1]['pressure'], 'humidity': x[1]['humidity'], 'humidity': x[1]['visibility'], 'precipIntensity': x[1]['precipIntensity'], 'precipProbability': x[1]['precipProbability']}

In [40]:
len(weatherDict)

199080

In [51]:
X = []
Y = []
for k in weatherDict.keys():
    if k in crimeDictionary:
        Y.append(1)
    else:
        Y.append(0)
#     X.append(weatherDict[k]['clock'])
    X.append([weatherDict[k]['clock'], float(weatherDict[k]['temp']), float(weatherDict[k]['pressure'])  ])

In [55]:
X_new = X[0:20000]
Y_new = Y[0:20000]

from sklearn import svm
clf = svm.SVC()
clf.fit(X_new, Y_new)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0, degree=3, gamma=0.0,
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [64]:
# y_new_prime = clf.predict(X_new[20000:21000])
clf.predict(X[20000:100000])

0

In [125]:
from sklearn.linear_model import LogisticRegression
X_train = X[0:150000]
Y_train = Y[0:150000]

logClf = LogisticRegression()
logClf.fit(X_train, Y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0)

In [126]:
P = (logClf.predict(X[150000:]))
#logClf.predict_proba(X[20000:100000])

In [160]:
def getAccuracy(Preds, Y, starter=0):
    falsePositive = 0
    falseNegative = 0
    truePositive = 0
    trueNegative = 0
    for i in xrange(len(Preds)):
        j = starter + i
        if Preds[i] == Y[j]:
            if Preds[i] == 1:
                truePositive += 1
            elif Preds[i] == 0:
                trueNegative += 1
            else:
                print Preds[i]
        else:
            if Preds[i] is 1:
                falsePositive += 1
            else:
                falseNegative += 1
    print "The True Positive Rate is : ", truePositive
    print "The True Negative Rate is : ", trueNegative
    print "The False Positive Rate is : ", falsePositive
    print "The False Negative Rate is : ", falseNegative
    return truePositive, trueNegative, falsePositive, falseNegative

In [161]:
tp, tn, fp, fn = getAccuracy(P, Y, starter=150000)

The True Positive Rate is :  458
The True Negative Rate is :  38624
The False Positive Rate is :  0
The False Negative Rate is :  9998


In [162]:
# 50% Threshold Accuracy
float((tn + tp))/(tp + tn + fp + fn)

0.7962917685411572

In [163]:
ProbPreds = logClf.predict_proba(X[150000:])

In [176]:
def thresholdPredictions(threshold, probs):
    return map(lambda x: 1 if x[1] > threshold else 0, probs)

In [169]:
# fortyPercentThresholdPreds
tp, tn, fp, fn = getAccuracy(fortyPercentThresholdPreds, Y, 150000)

The True Positive Rate is :  458
The True Negative Rate is :  38624
The False Positive Rate is :  1585
The False Negative Rate is :  8413


In [170]:
float((tn + tp))/(tp + tn + fp + fn)

0.7962917685411572

In [171]:
len(fortyPercentThresholdPreds)

49080

In [179]:
best_accuracy = 0
best_t = 0
for t in np.arange(0.1, 0.98, 0.2):
    t_p = thresholdPredictions(t, ProbPreds)
    tp, tn, fp, fn = getAccuracy(t_p, Y, 150000)
    accuracy = float((tn + tp))/(tp + tn + fp + fn)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_t = t
print best_accuracy
print best_t
print "########################"
print getAccuracy(thresholdPredictions(best_t, ProbPreds), Y, 150000)

The True Positive Rate is :  8871
The True Negative Rate is :  22402
The False Positive Rate is :  17807
The False Negative Rate is :  0
The True Positive Rate is :  4142
The True Negative Rate is :  31947
The False Positive Rate is :  8262
The False Negative Rate is :  4729
The True Positive Rate is :  458
The True Negative Rate is :  38624
The False Positive Rate is :  1585
The False Negative Rate is :  8413
The True Positive Rate is :  0
The True Negative Rate is :  40209
The False Positive Rate is :  0
The False Negative Rate is :  8871
The True Positive Rate is :  0
The True Negative Rate is :  40209
The False Positive Rate is :  0
The False Negative Rate is :  8871
0.819254278729
0.7
########################
The True Positive Rate is :  0
The True Negative Rate is :  40209
The False Positive Rate is :  0
The False Negative Rate is :  8871
(0, 40209, 0, 8871)


[12.0]
